In [ ]:
import os
import itertools
import json
from distutils.dir_util import copy_tree

# --- External Library Imports ---
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.ops import nearest_points
from shapely.geometry import Point, shape
import shapely

# --- Module Imports ---
import agent_config as config
import helper
import agent_sampling as samp

pd.options.mode.chained_assignment = None

In [ ]:
agents = pd.read_csv(os.path.join('india_base','agent_core_attributes.csv'), index=False))
agents.head()

In [ ]:
# --- convert all h5 files to pickles ---
# with h5py.File("pv_2014.h5", 'r+') as h5:
#     tables = list(h5.keys())

#     for table in tables:
#         if 'profile' in table:
#             profile = pd.DataFrame(h5[table][:])
#             profile.to_pickle(os.path.join('reference_data', 'solar_resource_pickles',f'{table}.pkl'))

# --- load resource meta table ---
meta = pd.read_pickle(os.path.join('reference_data', 'solar_resource_pickles','meta.pkl'))

# --- find tilt ---
def find_tilt(lat, array=[15,25,35,45,55]):
    array = np.asarray(array)
    idx = (np.abs(array - lat)).argmin()
    return array[idx]
meta['tilt'] = meta['latitude'].apply(find_tilt) #convert tilt to file terms
meta['azimuth'] = 180

# --- create centroid of each available resource file ---
meta['geometry'] = [Point(row['longitude'], row['latitude']) for _, row in meta.iterrows()]
meta = gpd.GeoDataFrame(meta)
meta.index.name = 'resource_id'
meta.reset_index(drop=False, inplace=True)
resource_points = meta.unary_union

# --- create list of centroids for each district ---
resource = agents.drop_duplicates(subset=['district_id'], keep='first')
resource['geometry'] = resource['centroid']
resource = resource[['geometry','district_id']]

try: #if column is text
    resource['geometry'] = resource['geometry'].apply(lambda x: shapely.wkt.loads(x))
except Exception as e:
    pass

resource = gpd.GeoDataFrame(resource)

# --- find nearest resource region for each district ---
def nearest_point_worker(point, points_lookup=resource_points, thresh=2):
    
    _, match = nearest_points(point, points_lookup)
    dist = point.distance(match)
    
    if dist < thresh:
        return match
    
    else:
        print('nearest point worker failed on', point.x, point.y)
        return np.nan

resource['geometry'] = resource['geometry'].apply(nearest_point_worker)

# --- merge district with meta ---
resource = resource.merge(meta[['resource_id','tilt','azimuth','geometry']], on='geometry')

# --- load resource data for each district ---
def load_resource_data(row):
    fp = os.path.join('reference_data', 'solar_resource_pickles',f"pv_a{row['azimuth']}_t{row['tilt']}_cf_profile.pkl")
    cf = pd.read_pickle(fp)
    profile = np.array(cf[row['resource_id']], dtype='int16')
    return profile

resource['cf'] = resource.apply(load_resource_data, axis=1)

resource = resource[['district_id','resource_id','tilt','azimuth','cf']]